<a id="RXNorm API"></a>
# RXNorm API

This notebook was prepared by Erik Nyquist (enyquis1) for use in EN.705.603.81.FA22.

This notebook explores the functionality of `RXNormAPI`. `RXNormAPI` queries the [RXNorm API](https://lhncbc.nlm.nih.gov/RxNav/APIs/RxNormAPIs.html?_gl=1*r0w82g*_ga*MTIxNzQzMTQxMS4xNjY4MzA5MzQ4*_ga_P1FPTH9PL4*MTY2ODMwOTM0Ny4xLjAuMTY2ODMwOTM0Ny4wLjAuMA..) to map a drug name to its RXNorm CUI identification code.

Let's instansiate an instance of `RXNormAPI` and demonstrate some core capabilities.

In [1]:
import sys
sys.path.append("..")

In [3]:
from medication_extractor.rxnorm_api import RXNormAPI

query = RXNormAPI().get_approximate_match("advil")
query

{'approximateGroup': {'inputTerm': None,
  'candidate': [{'rxcui': '153010',
    'rxaui': '1003945',
    'score': '11.108091354370117',
    'rank': '1',
    'source': 'MMSL'},
   {'rxcui': '153010',
    'rxaui': '1003946',
    'score': '11.108091354370117',
    'rank': '1',
    'name': 'Advil',
    'source': 'RXNORM'},
   {'rxcui': '153010',
    'rxaui': '12774003',
    'score': '11.108091354370117',
    'rank': '1',
    'name': 'Advil',
    'source': 'MSH'}]}}

This is the output from `RXNormAPI`. We are specifically interested in the "rxcui" code in the second dictionary where "source" is equal to "RXNROM". Let's extract that.

In [4]:
candiates = query['approximateGroup']['candidate']
rxnorm_val = next((item for item in candiates if item['source'] == "RXNORM"), None)
rxnorm_val

{'rxcui': '153010',
 'rxaui': '1003946',
 'score': '11.108091354370117',
 'rank': '1',
 'name': 'Advil',
 'source': 'RXNORM'}

In [5]:
rxnorm_val["rxcui"]

'153010'

Now let's package this process up for easy reuse.

In [6]:
# standard libaries
from typing import Dict, List


def map_medications(medications: List[str]) -> Dict[str, str]:
    """Map a list of medication extracted by `MedicationExtractor` 
    to their RXNorm CUI codes

    Args:
        medications (List[str]): List of medications

    Returns:
        Dict[str, str]: Mapping {Medication: RXCUI}
    """
    mapping_dict = {}
    
    for medication in medications:
        results = RXNormAPI().get_approximate_match(medication)
        candiates = results['approximateGroup']['candidate']
        rxnorm_val = next((item for item in candiates if item['source'] == "RXNORM"), None)
        mapping_dict[rxnorm_val["name"]] = rxnorm_val["rxcui"]

    return mapping_dict

In [7]:
map_medications(["advil"])

{'Advil': '153010'}

In [8]:
map_medications(["advil", "tylenol"])

{'Advil': '153010', 'Tylenol': '202433'}

In [10]:
map_medications(['midazolam', 'percocet', 'Aleve'])

{'midazolam': '6960', 'Percocet': '42844', 'Aleve': '215101'}

Now, after medications have been extracted by `MedicationExtractor`, we have a nice way to retrieve their RXNorm CUI codes from the database.